### The purpose of this notebook
1. Form dataset for training (provided training script) 
1. Auxiliary sets for problem: Frequency counts, dependency parsing, etc
2. Some definitions of problem
3. Simple baseline BIlstm + CRF for problem

### IGNORE/unorganized section

In [13]:
def find_approximate_occurance_based_on_structure(): #computes similarity between two sentences based on word structure and word contribution
    '''Example:
    s1: It is very overpriced and not very tasty
    s2: This lacks the features and is very expensive.
    s3: This lacks the features and is very expensive for a model A laptop.
    s4: It is very overpriced and not very tasty for a Michelin star restaraunt. 
    overpriced= expensive
    tasty = feaatures
    very = very
    not = lack
    Michelin star = model A 
    
    Domain dependent words--> get_domain_dependent_words()
    Domain independent words --> get_independent_words()
    
    'very expensive' is together/combined word 
    '''
    None
    
def find_approximate_occurance_based_on_words():
    '''Example:
    
    '''
    None
    
def find_inter_domain_similarity():
    

IndentationError: expected an indented block (<ipython-input-13-cf17692c7aec>, line 27)

#### Approach:

1. Baseline simple method: BiLSTM + CRF.  This is a baseline model. We need a model with high coverage (high false positives) and then trim the selection using the below model. This can either be simple rules, syntactic structures, just any model with high number of false positives. 


2. Given a sentence S, train an encoder-decoder network to reproduce the sentence. Compute last hidden state representation based on model M (S-> S). After training for a while, for each sentence: for each aspect word: remove BA, IA labelled words, and get representation. 2nd model takes this representation and the original sentence + some noisy cases and outputs a 1 or 0 depending on task. Effectively, 2nd model is learning to use incomplete representation and other info to learn whether it is an aspect/opinion term. 


To find out word contribution, by training a model based on missing words. So, if a word is removed, get some representation. Embed the ones with aspects missing to a similar space. Meaning if I have a sentence: I love their ice-cream so much. Then remove ice-cream, get a representation of the sentence, and a representation of the normal (with ice-cream sentence). Using these representations, map as similar (1) in a Siamese net. Now take all other (except opinion terms) and use same model to get representation. But the final model outputs the representations as dissimilar.

    G(F(I love their so much), F(I love their ice cream so much)) = 1
    G(F(love their ice cream so much), F(I love their ice cream so much)) = 0
    G(F(I love their ice cream), F(I love their ice cream so much)) = 2
    G(F(I love ice cream so much), F(Sentence)) = 



        Another approach could be if we remove the word then from all the mined words, if we replace the term with it, does the sentence still make sense (is it legible)-> then it is an aspect word. 

        So given s= I love their ice cream so much, if we remove ice-cream then replace it with say 50 previous aspect words like -> football, passing, nachos, ambience, does it still make sense? A model M is trained to identify whether a sentence still makes sense. 


#### What is an aspect and what is an opinion term?

S1: 'Plus it is small and reasonably light so I can take it with me to and from work. -> small, reasonably light

S2:'If internet connectivity is important I would recommend going with a dell net book for 50 bucks more, or buy a USB wireless card.' -> recommend, and internet connectivity;USB wireless card

S3: "Rao is a good restaurant, but it's nothing special." -> good, nothing special

S4: iLife is easily compatible with Microsoft Office so you can send and receive files from a PC.--> easily compatible-->  iLife;Microsoft Office 

S5: "people are rude bit again it's new york!" -> rude; people

S6: "The speed is incredible and I am more than satisfied." --> incredible, 'more than satisfied', 'speed'





In [14]:
'''To do:
Make baseline Bi-directional LSTM and top-CRF model with info about POS tags 

Inputs: POS_tags; Dependency relation wrt next word; Word vector (context info); Info about influence of a word 

Q: To get a better representation what can we do to the architecture itself (rather than feeding in more features)
Q: How do we obtain the influence of a word in a sentence (like on a color scale); attention
Q: How do we obtain the interdependeny of words -> dependency structure OR POSE OF WORDS
Q: Define interdependency with an example
'''

'To do:\nMake baseline Bi-directional LSTM and top-CRF model with info about POS tags \n\nInputs: POS_tags; Dependency relation wrt next word; Word vector (context info); Info about influence of a word \n\nQ: To get a better representation what can we do to the architecture itself (rather than feeding in more features)\nQ: How do we obtain the influence of a word in a sentence (like on a color scale); attention\nQ: How do we obtain the interdependeny of words -> dependency structure OR POSE OF WORDS\nQ: Define interdependency with an example\n'

In [15]:
word_to_int = {}
int_to_word = {}
word_vectors = [] 
pos_tagged_sentences = []
dependency_structure_sentences = []
pos_to_int = {}
label_to_int = {}


#for each training example
#concatenate word vector with pos tag and dependency structure



#1) Make word_to_id and id_to_word
#2) Tag each sentence as per scheme
#3) Use pytorch bilstm crf model initialized with word embeddings as a variable
#4) Train model to predict tags and make it batch wise (fine tune it)
#5) Explore adaptation 

In [16]:
#1) Convert data to format [([training tokens],[corresponding tags])..]
start_tag = "<START>"
end_tag = "<END>"
tag_to_id = {start_tag:0,end_tag:-1, "BA":1, "IA":2, "BO":3, "IO":4, "OT":5}
id_to_tag = {id_: tag for tag,id_ in tag_to_id.items()}
#construct token to id 
#use a tokenizer that adapts to most domains-> most of them are
#for sentence in laptop_data:

In [17]:
s = "U.K. is so so awesome, right!".translate(None, string.punctuation)

## Getting dataset info (do not rerun-already stored as pickle)

In [27]:

'''SCript to store dataset info'''
#https://explosion.ai/blog/sense2vec-with-spacy
#Does punctuation and case matter in this case? --> currently we do punctuation removal
import csv 
import string
import pickle
import pandas as pd
import nltk 
import spacy
import string

nlp = spacy.load('en') #load spacy model

start_tag = "<START>"
end_tag = "<END>"
tag_to_id = {start_tag:0,end_tag:-1, "BA":1, "IA":2, "BO":3, "IO":4, "OT":5}
id_to_tag = {id_: tag for tag,id_ in tag_to_id.items()}


def obtain_tokens(sentence):
    return sentence.split()



'''The below functions that use Spacy can be made faster using Spacy specific syntax- example using doc, checking for verbs, etc. However, here it is converted to Python string format to make the primary function compatible with other NLP packages'''
def get_tokenized_list_for_delim(phrases, tokenizer_func = obtain_tokens, delim =";"):
    '''
    Input: A sequence of phrases delimited as provided by delimiter: Ex A B; C; D E F
    Output: A list of tokenized phrases: [[A, B], [C], [D, E, F]]
    '''
    #split_phrases = phrases.split(delim)
    
    split_phrases = phrases.split(delim)
    tokenized_split_phrases = map(lambda x: tokenizer_func(x), split_phrases)
    return tokenized_split_phrases


def get_pos_tags(sentence):
    #Using spacy
    text = nlp(unicode(sentence))
    pos_tagged_text = [(str(token), str(token.pos_), str(token.tag_), str(token.dep_)) for token in text]
    return pos_tagged_text

def get_dependency_tree(sentence):
    #Using spacy again
    None
    


    
def get_dataset_info(df, save_to_file = False, save_dir ='laptop', include_sense = False, tokenizer_func=obtain_tokens, get_tags = True, remove_punctuation = False, replace_punctuation= True, to_lower = True, get_dependency_structure = False):
    training_data = []
    training_data_with_other_stuff = []
    out_row = []
    token_to_freq = {} 
    #token_sense_to_id = {}
    
    token_to_id = {}
    loop_i = 0 
    for sentence, aspect_words, opinion_words in zip(df.Sentence, df.Aspects, df.Opinions):
        
        
        if(remove_punctuation):
            sentence = sentence.translate(None, string.punctuation)
        
        
        if(to_lower):
            sentence = sentence.lower()
            
        
        tokenized_sentence = tokenizer_func(sentence)
        
        if(get_dependency_structure):
            #Obtain dependency tree
            dependency_tree = get_dependency_tree(sentence)
        #if(get_parser_tree):
            #Obtain parser tree
         #   None
        
        if(get_tags or include_sense):
            tagged_text = get_pos_tags(sentence) #NOTE: We do not give the tokenized sentence, only the sentence
        
        '''Before we replace punctuation with specific tags we need to perform all functions(POS, etc)'''
        if(replace_punctuation):
            
        
        
        aspect_list = get_tokenized_list_for_delim(aspect_words, tokenizer_func, ';')
        opinion_list = get_tokenized_list_for_delim(opinion_words, tokenizer_func, ';')
        #print(aspect_list)
        seq_absa_tagged = [] 
        aspect_ptr = 0
        opinion_ptr = 0
        skip = 0
        
        #labels = []
        for i_loop, token in enumerate(tokenized_sentence):
            temp_token = token 
            if(include_sense):
                '''THIS is specific to spacy format'''
            
                #Only limit to nouns, verbs and adjectives
                
                if(tagged_text[i_loop][1] in ['NOUN','VERB','ADJ']):
                    token+= '|'+ tagged_text[i_loop][1]
                    #print(token)
                    
            if token not in token_to_id:
                token_to_id[token] = len(token_to_id)
                token_to_freq[token] = 0 
            else:
                token_to_freq[token] += 1
                
                
                
            if(skip>0): #if we encounter incomplete aspect/opinion matches previously
                skip -= 1
             
            else:    
                label = [tag_to_id["OT"]] #assume it is OT, store as list in case of multiple aspect/opinion terms
        
        
                token = temp_token
                
                if(aspect_ptr < len(aspect_list) and token == aspect_list[aspect_ptr][0]): #Incomplete match: battery--> battery life
                    label = [tag_to_id["BA"]]
                    skip = len(aspect_list[aspect_ptr]) - 1 #words to skip ahead since they have been already covered
                    if(skip>0):
                        label += [tag_to_id["IA"] for i in aspect_list[aspect_ptr][1:]] 
                    aspect_ptr += 1 
        
                elif(opinion_ptr< len(opinion_list) and token == opinion_list[opinion_ptr][0]): 
                    label = [tag_to_id["BO"]]
                    skip = len(opinion_list[opinion_ptr]) - 1
                    if(skip>0):
                        label += [tag_to_id["IO"] for i in opinion_list[opinion_ptr][1:]]
                    opinion_ptr += 1 
        
            
                seq_absa_tagged+=label
                
        if(get_tags and get_dependency_structure):
            training_data_with_other_stuff.append((tokenized_sentence, seq_absa_tagged, tagged_text, dependency_tree))
            out_row = ["Sentence", "Sequence", "Tags","Dependency Tree"]
        
        elif(get_tags):
            training_data_with_other_stuff.append((tokenized_sentence, seq_absa_tagged, tagged_text))
            out_row = ["Sentence", "Sequence", "Tags"]
        
        elif(get_dependency_structure):
            training_data_with_other_stuff.append((tokenized_sentence, seq_absa_tagged, dependency_tree))
            out_row = ["Sentence", "Sequence", "Dependency Tree"]
        else:
            training_data_with_other_stuff.append((tokenized_sentence, seq_absa_tagged))
            out_row = ["Sentence", "Sequence"]
            
        training_data.append((tokenized_sentence, seq_absa_tagged))
        #Write data to csv and save vocab as pickle
        
        if(loop_i%300==0):
            print("At sentence: {}".format(-loop_i))
            print(training_data_with_other_stuff[-loop_i])
        loop_i-=1
    
    
    if(save_to_file):
        if(include_sense):
            save_dir+='_WITH_SENSE_'
        with open("Final_data/{}_absa_seq_labelled.csv".format(save_dir),'wb') as fout:
            csv_out = csv.writer(fout)
            csv_out.writerow(["Sentence","Sequence"])
            for row in training_data:
                csv_out.writerow(row)
    
        with open("Final_data/{}_additional_info_seq_labelled.csv".format(save_dir),'wb') as fout:
            csv_out = csv.writer(fout)
            csv_out.writerow(out_row)
            for row in training_data_with_other_stuff:
                csv_out.writerow(row) 
            
        with open("Final_data/{}_normal_training_list.pickle".format(save_dir),'wb') as pickle_o:
            pickle.dump(training_data, pickle_o)
        
        with open("Final_data/{}_additional_training_list.pickle".format(save_dir),'wb') as pickle_o:
            pickle.dump(training_data_with_other_stuff, pickle_o)
        
        with open("Final_data/{}_vocab.pickle".format(save_dir),'wb') as pickle_o: 
            pickle.dump(token_to_id, pickle_o)
        
        with open("Final_data/{}_tag2id.pickle".format(save_dir), 'wb') as pickle_o:
            pickle.dump(tag_to_id, pickle_o)
    
    return training_data_with_other_stuff, token_to_id
        


In [30]:
rest_data = pd.read_csv("Final_data/Semeval_14_ver1/Combined_restaurant.csv")
laptop_data = pd.read_csv("Final_data/Semeval_14_ver1/Combined_laptop.csv")

#laptop_training_data, token_to_id = get_dataset_info(laptop_data, True, 'laptop_lower', obtain_tokens, True )

In [31]:
laptop_tr_data, _ = get_dataset_info(laptop_data, True, 'laptop/',True, obtain_tokens, True)

At sentence: 0
(['i', 'charge', 'it', 'at', 'night', 'and', 'skip', 'taking', 'the', 'cord', 'with', 'me', 'because', 'of', 'the', 'good', 'battery', 'life'], [5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, 5, 5, 3, 1, 2], [('i', 'PRON', 'PRP', 'nsubj'), ('charge', 'VERB', 'VBP', 'ROOT'), ('it', 'PRON', 'PRP', 'dobj'), ('at', 'ADP', 'IN', 'prep'), ('night', 'NOUN', 'NN', 'pobj'), ('and', 'CCONJ', 'CC', 'cc'), ('skip', 'VERB', 'VB', 'conj'), ('taking', 'VERB', 'VBG', 'xcomp'), ('the', 'DET', 'DT', 'det'), ('cord', 'NOUN', 'NN', 'dobj'), ('with', 'ADP', 'IN', 'prep'), ('me', 'PRON', 'PRP', 'pobj'), ('because', 'ADP', 'IN', 'prep'), ('of', 'ADP', 'IN', 'pcomp'), ('the', 'DET', 'DT', 'det'), ('good', 'ADJ', 'JJ', 'amod'), ('battery', 'NOUN', 'NN', 'compound'), ('life', 'NOUN', 'NN', 'pobj')])
At sentence: 300
(['only', 'thing', 'i', 'would', 'want', 'to', 'add', 'to', 'this', 'is', 'an', 'internal', 'bluray', 'readwrite', 'drive'], [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 2, 2], [('only', 'ADJ', 'J

IOError: [Errno 2] No such file or directory: 'Final_data/laptop/_WITH_SENSE__absa_seq_labelled.csv'

## Frequency count of tags vs category

In [12]:
'''Script to get frequencies of POS tags for opinion and aspect words'''
from nltk import word_tokenize, pos_tag
from nltk.data import load
#tagdict = load('help/tagsets/upenn_tagset.pickle')

examples_of_aspect_as_ADJ = []
examples_of_opinion_as_NOUN = []


def get_freq_pos_opinion_and_aspect(df, limit =-1):
    '''Input is a dataframe'''
    #bopinion_tags = {tag:0 for tag in tagdict.keys()}
    #baspect_tags = {tag:0 for tag in tagdict.keys()}
    #iopinion_tags = {tag:0 for tag in tagdict.keys()}
    #iaspect_tags = {tag:0 for tag in tagdict.keys()}
    bopinion_tags = {} 
    baspect_tags = {}
    iopinion_tags = {} 
    iaspect_tags = {}
    '''
    if(limit==-1):
        limit = len(df)
        print_it = False
    else:
        limit = limit #for debugging
        '''
    for sentence, opinion, aspect in zip(df.Sentence, df.Opinions, df.Aspects):
        '''
        if(limit<=0):
            break
        limit-=1
        '''
        '''NLTK
        #text = word_tokenize(sentence) #<-NLTK
        #pos_tagged_text = pos_tag(text) #<-NLTK
        '''
        text = nlp(unicode(sentence))
        pos_tagged_text = [(str(token), str(token.pos_)) for token in text]
        
        
        for seqs in opinion.split(';'): #find all opinion terms-> can be done together with aspect terms but cleaner this way
            for i, opinion_term in enumerate(seqs.split()):
                for token_pos_tuple in pos_tagged_text:
                    token, pos_tag = token_pos_tuple[0], token_pos_tuple[1]
                    if(opinion_term == token):
                        if(i>1):#IO
                            if(pos_tag not in iopinion_tags):
                                iopinion_tags[pos_tag] = 1
                            else:
                                iopinion_tags[pos_tag] += 1
                        else:#BO
                            if(pos_tag not in bopinion_tags):
                                bopinion_tags[pos_tag] = 1
                            else:
                                bopinion_tags[pos_tag] += 1
                        '''    
                        if(limit> 0 and print_it):
                            print(sentence, token_pos_tuple)
                        '''
                        if(pos_tag == "NOUN"):
                            examples_of_opinion_as_NOUN.append((sentence, token_pos_tuple))
                        
        for seqs in aspect.split(';'): #find all aspect terms
            for i, aspect_term in enumerate(seqs.split()):
                for token_pos_tuple in pos_tagged_text:
                    token, pos_tag = token_pos_tuple[0], token_pos_tuple[1]
                    if(aspect_term == token):
                        if(i>1):#IA
                            if(pos_tag not in iaspect_tags):
                                iaspect_tags[pos_tag] = 1
                            else:
                                iaspect_tags[pos_tag] += 1
                        else:#BA
                            if(pos_tag not in baspect_tags):
                                baspect_tags[pos_tag] = 1
                            else:
                                baspect_tags[pos_tag] += 1
                       
                        if(pos_tag == 'ADJ'):
                            examples_of_aspect_as_ADJ.append((sentence, token_pos_tuple))
    return bopinion_tags, iopinion_tags, baspect_tags, iaspect_tags

#laptop_df = pd.read_csv("./Final_data/Semeval_14_ver1/Combined_laptop.csv")
#laptop_freq_dicts = get_freq_pos_opinion_and_aspect(laptop_df)

#rest_df = pd.read_csv("./Final_data/Semeval_14_ver1/Combined_restaurant.csv")
#rest_freq_dicts = get_freq_pos_opinion_and_aspect(rest_df)

KeyError: '<START>'

### Loading dataset info and embeddings

In [ ]:

#Load relevant data for training
def get_dataset_info(file_name = "laptop", training_data_additional = False, get_inverse = False, lower = True):
    
    if(lower):
        opt_string = "lower" #hardcoded
    else:
        opt_string = "final"
        
        
    if(training_data_additional):
        training_data = pickle.load(open("Final_data/{}_{}_additional_training_list.pickle".format(file_name, opt_string)))
    else:
        training_data = pickle.load(open("Final_data/{}_{}_normal_training_list.pickle".format(file_name, opt_string)))
        
    tag2id = pickle.load(open("Final_data/{}_{}_tag2id.pickle".format(file_name, opt_string)))
    
    vocab2id = pickle.load(open("Final_data/{}_{}_vocab.pickle".format(file_name, opt_string)))
    
    if("<START>" in vocab2id.keys()): #assuming <START> and <END> always co-occur
        start_token = vocab2id["<START"]
        end_token = vocab2id["<END>"]
    else:
        start_token = vocab2id["<START>"] = len(vocab2id)
        end_token = vocab2id["<END>"] = len(vocab2id)
    
    if(not get_inverse):
        return training_data, tag2id, vocab2id, start_token, end_token
    else:
        id2vocab = {id_:token for token, id_ in vocab2id.items()}
        id2tag = {id_:tag for tag, id_ in tag2id.items()}
        
        return training_data, tag2id, vocab2id, start_token, end_token, id2vocab, id2tag

#Load pretrained embeddings function 
def load_embeddings(path_to_embeddings, word2id, embedding_dim = 25):
    '''
    Input: This only takes in w2v format. So convert all other embedding types/vectors to w2v format: 
    Output: Torch variable with embeddings only belonging to and indexed by word2id
    
    For glove:
    Use python -m gensim.scripts.glove2word2vec -i <GloVe vector file> -o <Word2vec vector file>
    as per https://radimrehurek.com/gensim/scripts/glove2word2vec.html
    
    
    W2V format is:
    <Num vectors> <dimensionality>
    <word1> <vector rep>
    <word2> <vector rep>
    ....and so on
    '''
    word_vectors = KeyedVectors.load_word2vec_format(path_to_embeddings, binary = False)
    
    
    corpus_embeddings = np.zeros((len(word2id), embedding_dim))
    #Should words not found in embedding file be random or zeros? 
    #Currently 0 and then they are trained anyway
    '''Should we use a try/catch block here so that words that are not in vocab force an exception?
    Or should we choose every word in vocab and see if it is in embedding vocab-> worst case |V|*|embedding_vocab|
    Or vice versa, same worst case but early stopping 
    Or use key retrieval so all words in embedding vocab are checked if they belong to dictionary key
    '''
    for word in word2id.keys():
        if word in word_vectors.vocab:
            corpus_embeddings[word2id[word]] = np.array(word_vectors[word])
        
    return torch.from_numpy(corpus_embeddings).float()


In [15]:
def prepare_tokenized_list(phrases, delim =";"):
    '''
    Input: A sequence of phrases delimited by provided by delimiter: Ex A B; C; D E F
    Output: A list of tokenized phrases: [[A, B], [C], [D, E, F]]
    '''
    #split_phrases = phrases.split(delim)
    try:
        split_phrases = phrases.split(delim)
        spacy_objs = [nlp(unicode(phrase)) for phrase in split_phrases]
        tokenized_split_phrases = map(lambda phrases: [tokens for tokens in phrases], spacy_objs)
    except:
        print phrases
    return tokenized_split_phrases

#def nltk_tokenized_list(seq, delim=";"):
    